<a href="https://colab.research.google.com/github/sundar-nallalagappan/Bert_Base_Binary_classification/blob/main/Spam_Ham_usingBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
print("sairam")

sairam


In [28]:
!pip install -U "tensorflow==2.8.*"
!pip install -U "tensorflow-text==2.8.*"
import tensorflow as tf
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
print(tf.__version__)

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

2.8.4


In [30]:
!python --version

Python 3.8.16


In [31]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [32]:
train = pd.read_csv("spam.csv", encoding='ISO-8859-1')
train.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [33]:
train = train[['v1', 'v2']]

col_dict = {'v1': 'labels', 'v2':'Text'}
train = train.rename(col_dict, axis=1)

train.head()

,labels,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [34]:
train['labels'].value_counts()

ham     4825
spam     747
Name: labels, dtype: int64

In [35]:
df_ham = train[train['labels'] == 'ham']
df_spam = train[train['labels'] == 'spam']

print(df_ham.shape, df_spam.shape)

(4825, 2) (747, 2)


In [36]:
df_ham = df_ham.sample(df_spam.shape[0])
print(df_ham.shape, df_spam.shape)

(747, 2) (747, 2)


In [37]:
train = pd.concat([df_ham, df_spam])
train['labels'].value_counts()

ham     747
spam    747
Name: labels, dtype: int64

* Not a good idea to downsmaple - but just for quick implementation have done brutforce downsampling

In [38]:
train['spam'] = train['labels'].apply(lambda x: 1 if x == 'spam' else 0)
train['spam'].value_counts()

0    747
1    747
Name: spam, dtype: int64

In [39]:
train.tail()

,labels,Text,spam
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,1
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,1
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,1
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,1
5567,spam,This is the 2nd time we have tried 2 contact u...,1


In [40]:
X_train, X_test, y_train, y_test = train_test_split(train['Text'], train['spam'], stratify=train['spam'])
print(f"{X_train.shape, y_train.shape}")
print(f"{X_test.shape, y_test.shape}")

((1120,), (1120,))
((374,), (374,))


Preprocess and model url from tensorflow hub

In [41]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder    = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [42]:
text = ['i love sai']
e = bert_preprocess(text)
e

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 18952,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

In [44]:
e.keys()

dict_keys(['input_type_ids', 'input_word_ids', 'input_mask'])

In [46]:
def get_sentence_embeddings(sentence):
  preprocessed_output = bert_preprocess(sentence)
  embeds = bert_encoder(preprocessed_output)
  return embeds

get_sentence_embeddings([
    "500$ discount. hurry up", 
    "can we play tomorrow"]
)

{'default': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
 array([[-0.8435168 , -0.5132726 , -0.8884572 , ..., -0.74748856,
         -0.75314736,  0.91964495],
        [-0.9198247 , -0.3276891 , -0.29137257, ..., -0.14430751,
         -0.71314085,  0.9317897 ]], dtype=float32)>,
 'encoder_outputs': [<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
  array([[[ 0.00458483,  0.02783404, -0.12029573, ...,  0.25409517,
           -0.09419195, -0.09845521],
          [ 1.1227233 , -0.30777168, -1.0142773 , ...,  0.13534713,
            1.0343854 ,  0.34770805],
          [-0.21036682, -0.20707306, -0.7530634 , ...,  0.7503102 ,
            1.0195563 ,  0.37875172],
          ...,
          [-0.09700812, -0.24238297,  0.54475486, ...,  0.29192388,
           -0.30374658, -0.05979169],
          [-0.18046317, -0.24360123,  0.41981837, ...,  0.3194066 ,
           -0.27123314, -0.15537634],
          [-0.05397668, -0.18610531,  0.4423007 , ...,  0.62264186,
           -0.52400076, -0.

In [48]:
e = get_sentence_embeddings([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
print('Similarity of banana & grapes', cosine_similarity([e['pooled_output'][0]], [e['pooled_output'][1]]))
print('Similarity of banana & elon musk', cosine_similarity([e['pooled_output'][0]], [e['pooled_output'][-2]]))
print('Similarity of bill gates & elon musk', cosine_similarity([e['pooled_output'][-1]], [e['pooled_output'][-2]]))

Similarity of banana & grapes [[0.99110866]]
Similarity of banana & elon musk [[0.8933631]]
Similarity of bill gates & elon musk [[0.9164166]]


Build Model - Using functional API

In [63]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess(text_input)
embed_out = bert_encoder(preprocessed_text)

l1 = tf.keras.layers.Dropout(0.1, name='Dropout')(embed_out['pooled_output'])
l2 = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l1)

model = tf.keras.Model(inputs=[text_input], outputs=[l2])

In [64]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [65]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [66]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=5)

Epoch 1/5
70/70 [==============================] - 695s 10s/step - loss: 0.6218 - accuracy: 0.6687 - val_loss: 0.5082 - val_accuracy: 0.8663
Epoch 2/5
70/70 [==============================] - 681s 10s/step - loss: 0.4680 - accuracy: 0.8313 - val_loss: 0.4156 - val_accuracy: 0.8770
Epoch 3/5
70/70 [==============================] - 671s 10s/step - loss: 0.4007 - accuracy: 0.8518 - val_loss: 0.3821 - val_accuracy: 0.8636
Epoch 4/5
70/70 [==============================] - 671s 10s/step - loss: 0.3569 - accuracy: 0.8866 - val_loss: 0.3350 - val_accuracy: 0.8850
Epoch 5/5
70/70 [==============================] - 672s 10s/step - loss: 0.3289 - accuracy: 0.8920 - val_loss: 0.3069 - val_accuracy: 0.8904


In [67]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 189s 15s/step - loss: 0.3069 - accuracy: 0.8904


[0.30687615275382996, 0.8903743028640747]

Inference

In [70]:
sms = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
y_hat = model.predict(sms)
y_hat

array([[0.7290125 ],
       [0.80359244],
       [0.7618607 ],
       [0.25509614],
       [0.14641207]], dtype=float32)

In [73]:
np.where(y_hat > 0.5, 1, 0)

array([[1],
       [1],
       [1],
       [0],
       [0]])